# __Fetch aoe2recs.com match details sequentially__

### __Load libraries__

In [1]:
from pymongo import MongoClient
from tqdm import tqdm
import requests
import json
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### __Source functions__

In [2]:
%run aoe2rec_api_modules.ipynb

### __Setup database connection__

In [3]:
db_conn = setup_mongo_conn()

### __Fetch last matchID downloaded__

In [4]:
match_start_id = fetch_latest_match_id_aoe2recs(db_conn) + 1

### __Fetch match details__

In [ ]:
fetch_matches_iterate_aoe2recs(match_start_id,10,db_conn)

      50%|██████████████████████████████                              | 5/10 [01:22<02:03, 24.76s/it]